# MoI ML CNN Experiment

- Train op fashion mnist
- Laat experimenten zien: kies 2 - 3 params, kies een range en voer tests uit. Doel is 93% accuracy.

In [23]:
import torch
import torch.nn as nn
import seaborn as sns
import sys
from pathlib import Path
sys.path.insert(0, "../")

In [24]:
from src.data import make_dataset
from src.models import imagemodels
from src.models import train_model
import gin


In [25]:
# Get cpu or gpu device for training.
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

Using cpu device


In [26]:
# Load config for this notebook
gin.parse_config_file("cnn.gin")

ParsedConfigFileIncludesAndImports(filename='cnn.gin', imports=[], includes=[])

In [27]:
# Get MNIST data
train_dataloader, test_dataloader = make_dataset.get_MNIST()

In [28]:

# Set some other parameters
import torch.optim as optim
from src.models import metrics
optimizer = optim.Adam
loss_fn = torch.nn.CrossEntropyLoss()
accuracy = metrics.Accuracy()
from src.models import train_model

## Voer een experiment uit.


### Configuratie voor het experiment

Sla de gewenste structuur voor de lagen op in een lijst configuraties

In [ ]:
from dataclasses import dataclass
import itertools 
@dataclass
class NNLayerConfig:
    """Class for storing config and results for a layer."""
    loss: float
    num_params: int
    c1_ksize: int
    c1_stride: int
    c2_ksize: int
    c2_stride: int
    num_filters: int

configs = []  # List of NNLayerConfig objects

# Configuration for the experiment. We ignore padding for now
kernel_sizes = [1, 2, 3]
filter_nums = [16, 32, 64]
strides = [1, 2, 3]
for config in itertools.product(kernel_sizes, # kernel size in convolution 1
                                strides, # stride size in convolution 1
                                kernel_sizes, # kernel size in convolution 2
                                strides, # stride size in convolution 2
                                filter_nums, # number of filters to use
                                ):
    configs.append(NNLayerConfig(
        loss = 0,
        num_params = 0,
        c1_ksize = config[0],
        c1_stride = config[1],
        c2_ksize = config[2],
        c2_stride = config[2],
        num_filters = int,
    ))


Maak een klasse die met een configuratie als input een NN maakt.

In [37]:



# Define model
class ConfigurableNN (nn.Module):

    def __init__(self, config:NNLayerConfig):
        super().__init__()

        self.config = config
        self.num_filters_l1 = 32 # TODO from config
        self.max_pool_ksize = 2 # TODO from config

# Calculating output size:
# Second method: (((W - K + 2P)/S) + 1)
#         Here W = Input size
#         K = Filter size
#         S = Stride
#         P = Padding 

        # Convolutions are set separately
        self.convolutions = nn.Sequential(
            nn.Conv2d(  1, # First one is always 1
                        self.num_filters_l1, 
                        kernel_size = config.filter1_size,
                        stride = config.filter1_stride,
                        padding = config.filter1_padding),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=self.max_pool_ksize),
            nn.Conv2d(  self.num_filters_l1,
                        self.num_filters_l1 * 2,
                        kernel_size = config.filter2_size,
                        stride = config.filter2_stride,
                        padding = config.filter2_padding),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=self.max_pool_ksize)

            # Werkend voorbeeld:
            # nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1),
            # nn.ReLU(),
            # nn.MaxPool2d(kernel_size=2),
            # nn.Conv2d(32, 32, kernel_size=3, stride=1, padding=0),
            # nn.ReLU(),
            # nn.MaxPool2d(kernel_size=2),
            # nn.Conv2d(32, 32, kernel_size=3, stride=1, padding=0),
            # nn.ReLU(),
            # nn.MaxPool2d(kernel_size=2),


        )
        
        # TODO Calculate the size that results from self convolutions
        # and input it into the first parameter of the first nn.Linear below.
        # # Dense is default for now.
        self.dense = nn.Sequential(
            nn.Flatten(),
            nn.Linear(2304, 64),
            nn.ReLU(),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Linear(32, 10)
        )

    # Forward is default for now.
    def forward(self, x):
        x = self.convolutions(x)
        logits = self.dense(x)
        return logits


Loop door de configuraties heen

In [46]:
def dump_model_steps(model, example_input):
    """
    List the steps of each convolution in a model and show the result for a given input. If possible, also show what happens in a dense layer
    """
    current = example_input
    print("===============")
    print(f'Starting model with shape {current.shape}')
    print("===============")
    if(model.convolutions):
        for conv in model.convolutions:
            print("-----------------")
            print("Convolution:")
            print(conv)
            current = conv(current)
            print(current.shape)
    if(model.dense):
        print("Contents of dense:")
        for dense in model.dense:
            print("-----------------")
            print("Dense:")
            print(dense)
            current = dense(current)
            print(current.shape)


In [45]:
num_epochs = 2 # During development keep very low. 10 is for real tests
learning_rate = 1e-3

# For to be debugging:
x, y = next(iter(train_dataloader))
x.shape, y.shape


for config in configs:
    model = ConfigurableNN(config).to(device)
    dump_model_steps(model, x)
    break
    model = train_model.trainloop(
        epochs=num_epochs,
        model=model,
        optimizer=optimizer,
        learning_rate=learning_rate,
        loss_fn=loss_fn,
        metrics=[accuracy],
        train_dataloader=train_dataloader,
        test_dataloader=test_dataloader,
        log_dir="../models/test/",
        train_steps=len(train_dataloader),
        eval_steps=len(test_dataloader),
    )
    break # During development
    

Starting model with shape torch.Size([32, 1, 28, 28])
Convolutions:
-----------------
Convolution:
Conv2d(1, 32, kernel_size=(1, 1), stride=(1, 1))
torch.Size([32, 32, 28, 28])
-----------------
Convolution:
ReLU()
torch.Size([32, 32, 28, 28])
-----------------
Convolution:
MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
torch.Size([32, 32, 14, 14])
-----------------
Convolution:
Conv2d(32, 64, kernel_size=(2, 2), stride=(1, 1))
torch.Size([32, 64, 13, 13])
-----------------
Convolution:
ReLU()
torch.Size([32, 64, 13, 13])
-----------------
Convolution:
MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
torch.Size([32, 64, 6, 6])
Contents of dense:
-----------------
Dense:
Flatten(start_dim=1, end_dim=-1)
torch.Size([32, 2304])
-----------------
Dense:
Linear(in_features=2304, out_features=64, bias=True)
torch.Size([32, 64])
-----------------
Dense:
ReLU()
torch.Size([32, 64])
-----------------
Dense:
Linear(in_features=64, out_feature

Checkout results using tensorboard: 
- run `tensorboard --logdir models` in the terminal
- tensorboard will launch at `localhost:6006` and vscode will notify you that the port is forwarded
- you can either press the `launch` button in VScode or open your local browser at `localhost:6006`